In [8]:
import sys
import getopt
import os
#from sets import Set
import json
import pprint
from graphviz import Digraph
from math import log
import src.searchtree as st
import src.utils as utils

In [9]:
#agent is the agent we are trying to compromise. We are the adversary. All other agents are the adversary.
agent=0
domain="logistics-small-factored"#"uav-factored"#""logistics-small-factored" #
problem="test"#"example-uavs"#"test"#"probLOGISTICS-4-0"

st.configuration["privateActions"]=True 
st.configuration["useFullStates"]=1
st.configuration["nTo1Mapping"]=1
st.configuration["SecureMAFS"]=False
st.configuration["debug"]=True
st.configuration["console"]=False

if st.configuration["console"]:
    #read options
    params = ["domain=","problem=","agent=","use-full-states=","n-to-1-mapping=","secure-mafs","debug"]
    #print(str(sys.argv))
    try:
        opts, args = getopt.getopt(sys.argv[1:],'',params)
        #print("opts:"+str(opts))
        #print("args:"+str(args))
    except getopt.GetoptError:
        print ('bad leakage.py params: ' + str(params))
        sys.exit(2)

    for opt, arg in opts:
        print("opt="+str(opt)+",arg="+str(arg))
        if opt == "--domain":
            domain = arg
        elif opt == "--problem":
            problem = arg
        elif opt == "--agent":
            agent = int(arg)
        elif opt == "--use-full-states":
            st.configuration["useFullStates"] = int(arg)
        elif opt == "--n-to-1-mapping":
            st.configuration["nTo1Mapping"] = int(arg)
        elif opt == "--secure-mafs":
            st.configuration["SecureMAFS"]=True
        elif opt == "--debug":
            st.configuration["debug"]=True

root="traces/"+domain+"/"+problem+"/"+str(st.configuration["useFullStates"])+"-"+str(st.configuration["nTo1Mapping"])
agentFile=root+"/agent"+str(agent)+".json"
#adversaryFile=root+"/agent"+str(adversary)+".json"

if st.configuration["SecureMAFS"]:
    outputFile=root+"-SecureMAFS/agent"+str(agent)
else:
    outputFile=root+"/agent"+str(agent)

outputCSVFile = "./results.csv"

In [10]:
#load data
varMap = {}
stateMap = {}
#states = {}
opMap = {}
operators = set()

advers = set()
states = []

plan = []


for fileName in os.listdir(root):
    agentID = -1
    if fileName.find("agent")!= -1 and fileName.find(".json")!= -1:
        #print("next: "+fileName[fileName.find("agent")+5:fileName.find(".json")])
        agentID=int(fileName[fileName.find("agent")+5:fileName.find(".json")])
        #print(agentID)
    if agentID != -1 and agentID != agent:
        print("processing " + fileName)
        
        advers.add(agentID)

        f = open(root+"/"+fileName)
        data = json.load(f)
        f.close()

        #load variables
        for v in data["variables"]:
            #print(v)
            var = st.Variable(v)
            varMap[var.hash] = var
        
        if st.configuration["debug"]:
            print("variables:")
            pprint.pprint(varMap)
            
        #load states
        order = 0
        secureMAFSStates = set()
        for s in data["states"]:
            state = st.State(s,varMap,order)
            
            #ignore states not sent by Secure-MAFS 
            #TODO: verify correctness
            if st.configuration["SecureMAFS"] and state.senderID == agent:
                if state.getSecureMAFSSignature() in secureMAFSStates:
                    continue
                else:
                    secureMAFSStates.add(state.getSecureMAFSSignature())
            
            stateMap[state.hash] = state
            #states[agentID].append(state)
            order += 1
            
        states = [s for s in stateMap.values()]

        #load operators (and convert to label non-preserving projection)
        allOps = filter(lambda x: x["ownerID"]==agent,data["operators"])
        
        for op in allOps:
            #print(op)
            operator = st.Operator(op)
            if operator.hash in opMap:
                opMap[operator.hash].process(operator)
            else:
                opMap[operator.hash] = operator

        operators = operators | set(opMap.values())
        
        plan = data["plan"]

        
print("done!")  
print("variables:" + str(len(varMap)))
print("operators:" + str(len(operators)))
print("states:" + str(len(states)))

if st.configuration["debug"]:
    print("varMap:")
    pprint.pprint(varMap)
    
    if len(states) < 25:
        print("stateMap:")
        pprint.pprint(stateMap)
        print("states:")
        pprint.pprint(states)
        
    print("opMap:")
    pprint.pprint(opMap)
    print("operators:")
    pprint.pprint(operators)
    print("plan:")
    pprint.pprint(plan)
            



processing agent1.json
variables:
{'1:1': 1:1:True:{'2': '<none of those>', '0': '(P)at(tru1, apt1)', '1': '(P)at(tru1, pos1)'},
 '1:2': 1:2:True:{'0': '(P)in(obj11, tru1)', '1': '(N)in(obj11, tru1)'},
 'P:0': P:0:False:{'2': 'at(obj11, pos1)', '3': '<none of those>', '0': 'at(obj11, apt1)', '1': 'at(obj11, apt2)'}}
processing agent2.json
variables:
{'1:1': 1:1:True:{'2': '<none of those>', '0': '(P)at(tru1, apt1)', '1': '(P)at(tru1, pos1)'},
 '1:2': 1:2:True:{'0': '(P)in(obj11, tru1)', '1': '(N)in(obj11, tru1)'},
 '2:1': 2:1:True:{'2': '<none of those>', '0': '(P)at(obj11, pos2)', '1': '(P)in(obj11, tru2)'},
 '2:2': 2:2:True:{'2': '<none of those>', '0': '(P)at(tru2, apt2)', '1': '(P)at(tru2, pos2)'},
 'P:0': P:0:False:{'2': 'at(obj11, pos1)', '3': '<none of those>', '0': 'at(obj11, apt1)', '1': 'at(obj11, apt2)'}}
done!
variables:5
operators:4
states:9
varMap:
{'1:1': 1:1:True:{'2': '<none of those>', '0': '(P)at(tru1, apt1)', '1': '(P)at(tru1, pos1)'},
 '1:2': 1:2:True:{'0': '(P)in(

In [11]:
#load agent data (instead of estimates)
f = open(agentFile)
agentData = json.load(f)
f.close()

agentPrivateVariables = 0
agentDomainSize = 0

for var in agentData["variables"]:
    if var["isPrivate"]:
        agentPrivateVariables += 1
        agentDomainSize = max(agentDomainSize,len(var["vals"]))
        
for op in agentData["operators"]:
    if op["isPrivate"]:
        st.configuration["privateActions"]=True
        break
        
print("agent private variables: " + str(agentPrivateVariables))
print("agent domain size: " + str(agentDomainSize))
print("private actions: " + str(st.configuration["privateActions"]))

agent private variables: 2
agent domain size: 3
private actions: True


In [12]:
#DEBUG: find states which have the i-parent the same as they are

suspicious = set()

print("states which refer to themselves as i-parents:")
for s in states:
    if s.privateIDs[s.agentID] == s.stateID:
        suspicious.add(s)
        print(s.printAll())
        
pprint.pprint(suspicious)

print("received states which refer non-existent states as i-parents:")
suspicious = set()
for s in states:
    if not s.isReceived():
        continue
        
    stateHash = str(s.agentID) + ":" + str(s.privateIDs[s.agentID])
    if not stateHash in stateMap:
        suspicious.add(s)
        print(s.printAll())
        print("i-parent: " + stateHash)

pprint.pprint(suspicious)
        
    

states which refer to themselves as i-parents:
{
	agentID=2
	senderID=-1
	stateID=0
	parentID=-1
	privateIDs=[0, 0, 0]
	publicValues=[2]
	privateValues=[2, 1]
	context=init
}
{
	agentID=1
	senderID=-1
	stateID=0
	parentID=-1
	privateIDs=[0, 0, 0]
	publicValues=[2]
	privateValues=[1, 1]
	context=init
}
{
	agentID=2
	senderID=2
	stateID=8
	parentID=-1
	privateIDs=[8, 4, 8]
	publicValues=[1]
	privateValues=[2, 1]
	context=goal-verified
}
{2:8:[1],[2, 1],[8, 4, 8], 2:0:[2],[2, 1],[0, 0, 0], 1:0:[2],[1, 1],[0, 0, 0]}
received states which refer non-existent states as i-parents:
{
	agentID=1
	senderID=0
	stateID=6
	parentID=-1
	privateIDs=[6, 2, 0]
	publicValues=[3]
	privateValues=[0, 1]
	context=received
}
i-parent: 1:2
{1:6:[3],[0, 1],[6, 2, 0]}
